In [ ]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving _0411_all_features_with_nans(target included).csv to _0411_all_features_with_nans(target included).csv


In [ ]:
import io
X = pd.read_csv(io.BytesIO(uploaded['_0411_all_features_with_nans(target included).csv']))

In [ ]:
X_num = X.copy()
X_num.drop(columns = ['Unnamed: 0', 'SubjectID','Gender','Race','family_ALS_hist','onset_site','if_use_Riluzole','treatment_group'], inplace = True)
X_cat = X[['SubjectID', 'Gender', 'Race', 'family_ALS_hist', 'if_use_Riluzole', 'treatment_group', 'onset_site']]
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SubjectID        1650 non-null   int64  
 1   Gender           1650 non-null   int64  
 2   Race             1650 non-null   object 
 3   family_ALS_hist  369 non-null    object 
 4   if_use_Riluzole  1247 non-null   float64
 5   treatment_group  1477 non-null   float64
 6   onset_site       1650 non-null   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 90.4+ KB


##Impute Categorical variable

In [ ]:
mm = MinMaxScaler()

def find_category_mappings(data, variable):
    return {k: i for i, k in enumerate(data[variable].dropna().unique(), 0)}

def integer_encode(data , variable, ordinal_mapping):
    data[variable] = data[variable].map(ordinal_mapping)

mappin = dict()
def imputation(data1 , cols):
    data = data1.copy()
    #Encoding dict &amp; Removing nan    
    #mappin = dict()
    for variable in cols:
        mappings = find_category_mappings(data, variable)
        mappin[variable] = mappings

    #Apply mapping
    for variable in cols:
        integer_encode(data, variable, mappin[variable])  

    #Minmaxscaler and KNN imputation 
    sca = mm.fit_transform(data)
    knn_imputer = KNNImputer()
    knn = knn_imputer.fit_transform(sca)
    data.iloc[:,:] = mm.inverse_transform(knn)
    for i in data.columns : 
        data[i] = round(data[i]).astype('int')

    #Inverse transform
    for i in cols:
        inv_map = {v: k for k, v in mappin[i].items()}
        data[i] = data[i].map(inv_map)
    return data


#Imputation
X_cat_imp = imputation(X_cat,['Gender', 'Race', 'family_ALS_hist', 'if_use_Riluzole', 'treatment_group', 'onset_site'])

In [ ]:
X_cat_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SubjectID        1650 non-null   int64  
 1   Gender           1650 non-null   int64  
 2   Race             1650 non-null   object 
 3   family_ALS_hist  1650 non-null   object 
 4   if_use_Riluzole  1650 non-null   float64
 5   treatment_group  1650 non-null   float64
 6   onset_site       1650 non-null   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 90.4+ KB


##Impute Numerical variable

In [ ]:
columns = X_num.columns
imputer = IterativeImputer(sample_posterior = True, random_state = 10)
ar = imputer.fit_transform(X_num)
X_num_imp = pd.DataFrame(ar, columns = columns)

In [ ]:
X_num_imp

,Age,diag_delta,onset_delta,fvc_slope,fvc_mean,svc_slope,svc_mean,mean_bp_d,mean_bp_s,mean_pulse,mean_resp_rate,mean_temp,bp_d_slope,bp_s_slope,pulse_slope,resp_rate_slope,temp_slope,mean_weight,slope_weight,mean_BMI,mean_ALC,mean_AMC,mean_ANC,mean_CK,mean_Creatinine,mean_Total_Cholesterol,mean_Triglycerides,mean_Uric_Acid,ALC_slope,AMC_slope,ANC_slope,CK_slope,Creaetinine_slope,Total_Cholesterol_slope,Triglycerides_slope,Uric_Acid_slope,mean_alsfrs_total,alsfrs_total_slope,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor,ALSFRS_slope
0,52.0,-302.0,-501.0,-0.931358,103.293488,-0.018609,81.666667,88.250000,137.000000,85.250000,17.000000,37.025000,-0.081967,-0.327869,0.114754,-0.065574,0.000000,120.033333,0.060656,0.004471,1.726124,0.385000,5.210000,199.750000,53.040000,4.971600,1.366100,295.913000,-0.016216,0.000889,-0.009667,1.200000,0.000000,-0.004310,0.005394,0.396533,30.250000,-0.009689,4.00,4.000000,4.00,3.50,2.00,2.250000,3.50,2.000000,1.000000,4.00,8.00,14.250000,0.000000,0.000000,0.000000,0.016393,0.000000,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393,-0.341760
1,65.0,-44.0,-1023.0,-0.583333,73.500000,0.223395,130.471062,83.000000,141.333333,69.166667,17.333333,35.825000,0.021978,-0.043956,0.109890,0.000000,-0.003297,51.000000,-0.005495,0.002187,0.573115,-0.091506,-2.202054,149.000000,79.560000,4.539395,3.205620,182.941528,-0.016199,0.014305,-0.008014,0.681319,0.000000,0.021458,0.043421,-0.369212,28.666667,0.002114,1.00,1.333333,2.00,4.00,4.00,3.333333,4.00,3.333333,2.666667,3.00,3.00,21.333333,-0.010989,-0.010989,0.000000,0.000000,0.000000,-0.010989,0.010989,0.010989,0.010989,0.000000,-0.010989,0.010989,-0.965608
2,66.0,-462.0,-550.0,0.076462,52.652720,-0.158016,82.666667,69.500000,122.000000,71.500000,20.500000,36.775000,0.028986,-0.144928,-0.086957,-0.057971,0.000000,80.700000,-0.020290,0.002810,2.034695,0.492000,4.466000,166.250000,70.720000,4.887540,1.919320,278.366400,-0.000993,-0.006957,0.018406,0.057971,0.000000,0.000374,0.001800,-0.258609,32.000000,-0.043752,0.75,2.750000,0.75,4.00,4.00,4.000000,4.00,4.000000,4.000000,3.75,1.50,24.000000,0.000000,-0.014493,-0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014493,-0.014493,0.000000,-0.569600
3,41.0,-511.0,-570.0,0.000000,71.000000,0.045068,90.613842,81.571429,125.000000,82.000000,17.666667,36.900000,0.157895,-0.052632,0.355263,0.052632,0.001316,121.400000,-0.036770,0.004191,2.404000,0.464000,5.462000,182.405118,58.400000,4.549952,2.478876,317.882655,0.003896,0.001948,0.060390,0.968414,0.000000,-0.030289,-0.019621,0.641531,16.500000,-0.104167,2.50,3.000000,3.00,1.50,1.00,1.000000,0.50,0.500000,0.000000,3.50,5.50,4.500000,-0.020833,-0.041667,0.000000,-0.020833,0.000000,0.000000,-0.020833,-0.020833,0.000000,0.020833,-0.020833,-0.062500,-1.183528
4,63.0,-209.0,-365.0,-0.235294,114.000000,-0.364164,114.042740,85.000000,140.000000,70.000000,16.000000,36.800000,-0.294118,-0.588235,-0.117647,0.000000,-0.017647,90.100000,0.000000,0.002976,2.297509,0.388903,2.152266,351.105529,66.300000,7.392401,0.899067,341.987231,0.014874,0.000550,0.003669,-0.207561,0.260000,-0.001873,-0.003815,-0.125311,30.000000,0.000000,4.00,4.000000,4.00,3.00,3.00,2.000000,3.00,3.000000,1.000000,3.00,8.00,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,-0.598361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

##Final imputed data

In [ ]:
X_imp = pd.concat([X_cat_imp, X_num_imp], axis = 1)
X_imp['diag_minus_onset'] = X_imp['diag_delta']- X_imp['onset_delta']
X_imp

,SubjectID,Gender,Race,family_ALS_hist,if_use_Riluzole,treatment_group,onset_site,Age,diag_delta,onset_delta,fvc_slope,fvc_mean,svc_slope,svc_mean,mean_bp_d,mean_bp_s,mean_pulse,mean_resp_rate,mean_temp,bp_d_slope,bp_s_slope,pulse_slope,resp_rate_slope,temp_slope,mean_weight,slope_weight,mean_BMI,mean_ALC,mean_AMC,mean_ANC,mean_CK,mean_Creatinine,mean_Total_Cholesterol,mean_Triglycerides,mean_Uric_Acid,ALC_slope,AMC_slope,ANC_slope,CK_slope,Creaetinine_slope,Total_Cholesterol_slope,Triglycerides_slope,Uric_Acid_slope,mean_alsfrs_total,alsfrs_total_slope,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor,ALSFRS_slope,diag_minus_onset
0,348,0,0,N,0.0,0.0,0,52.0,-302.0,-501.0,-0.931358,103.293488,-0.018609,81.666667,88.250000,137.000000,85.250000,17.000000,37.025000,-0.081967,-0.327869,0.114754,-0.065574,0.000000,120.033333,0.060656,0.004471,1.726124,0.385000,5.210000,199.750000,53.040000,4.971600,1.366100,295.913000,-0.016216,0.000889,-0.009667,1.200000,0.000000,-0.004310,0.005394,0.396533,30.250000,-0.009689,4.00,4.000000,4.00,3.50,2.00,2.250000,3.50,2.000000,1.000000,4.00,8.00,14.250000,0.000000,0.000000,0.000000,0.016393,0.000000,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393,-0.341760,199.0
1,533,0,0,N,0.0,0.0,1,65.0,-44.0,-1023.0,-0.583333,73.500000,0.223395,130.471062,83.000000,141.333333,69.166667,17.333333,35.825000,0.021978,-0.043956,0.109890,0.000000,-0.003297,51.000000,-0.005495,0.002187,0.573115,-0.091506,-2.202054,149.000000,79.560000,4.539395,3.205620,182.941528,-0.016199,0.014305,-0.008014,0.681319,0.000000,0.021458,0.043421,-0.369212,28.666667,0.002114,1.00,1.333333,2.00,4.00,4.00,3.333333,4.00,3.333333,2.666667,3.00,3.00,21.333333,-0.010989,-0.010989,0.000000,0.000000,0.000000,-0.010989,0.010989,0.010989,0.010989,0.000000,-0.010989,0.010989,-0.965608,979.0
2,1064,0,0,N,0.0,0.0,1,66.0,-462.0,-550.0,0.076462,52.652720,-0.158016,82.666667,69.500000,122.000000,71.500000,20.500000,36.775000,0.028986,-0.144928,-0.086957,-0.057971,0.000000,80.700000,-0.020290,0.002810,2.034695,0.492000,4.466000,166.250000,70.720000,4.887540,1.919320,278.366400,-0.000993,-0.006957,0.018406,0.057971,0.000000,0.000374,0.001800,-0.258609,32.000000,-0.043752,0.75,2.750000,0.75,4.00,4.00,4.000000,4.00,4.000000,4.000000,3.75,1.50,24.000000,0.000000,-0.014493,-0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014493,-0.014493,0.000000,-0.569600,88.0
3,1736,1,0,N,0.0,1.0,2,41.0,-511.0,-570.0,0.000000,71.000000,0.045068,90.613842,81.571429,125.000000,82.000000,17.666667,36.900000,0.157895,-0.052632,0.355263,0.052632,0.001316,121.400000,-0.036770,0.004191,2.404000,0.464000,5.462000,182.405118,58.400000,4.549952,2.478876,317.882655,0.003896,0.001948,0.060390,0.968414,0.000000,-0.030289,-0.019621,0.641531,16.500000,-0.104167,2.50,3.000000,3.00,1.50,1.00,1.000000,0.50,0.500000,0.000000,3.50,5.50,4.500000,-0.020833,-0.041667,0.000000,-0.020833,0.000000,0.000000,-0.020833,-0.020833,0.000000,0.020833,-0.020833,-0.062500,-1.183528,59.0
4,2492,1,0,N,0.0,1.0,2,63.0,-209.0,-365.0,-0.235294,114.000000,-0.364164,114.042740,85.000000,140.000000,70.000000,16.000000,36.800000,-0.294118,-0.588235,-0.117647,0.000000,-0.017647,90.100000,0.000000,0.002976,2.297509,0.388903,2.152266,351.105529,66.300000,7.392401,0.899067,341.987231,0.014874,0.000550,0.003669,-0.207561,0.260000,-0.001873,-0.003815,-0.125311,30.000000,0.000000,4.00,4.000000,4.00,3.00,3.00,2.000000,3.00,3.000000,1.000000,3.00,8.00,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,-0.598361,156.0
...

##Concatenate feature data & survival data

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving survival.csv to survival.csv


In [ ]:
import io
surv = pd.read_csv(io.BytesIO(uploaded['survival.csv']))

In [ ]:
extent = list(surv['SubjectID'].unique())
X_imp_filt = X_imp.query("SubjectID == {0}".format(extent))
X_imp_surv = X_imp_filt.merge(surv, how = 'inner')
X_imp_surv

,SubjectID,Gender,Race,family_ALS_hist,if_use_Riluzole,treatment_group,onset_site,Age,diag_delta,onset_delta,fvc_slope,fvc_mean,svc_slope,svc_mean,mean_bp_d,mean_bp_s,mean_pulse,mean_resp_rate,mean_temp,bp_d_slope,bp_s_slope,pulse_slope,resp_rate_slope,temp_slope,mean_weight,slope_weight,mean_BMI,mean_ALC,mean_AMC,mean_ANC,mean_CK,mean_Creatinine,mean_Total_Cholesterol,mean_Triglycerides,mean_Uric_Acid,ALC_slope,AMC_slope,ANC_slope,CK_slope,Creaetinine_slope,Total_Cholesterol_slope,Triglycerides_slope,Uric_Acid_slope,mean_alsfrs_total,alsfrs_total_slope,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor,ALSFRS_slope,diag_minus_onset,time_event,status
0,348,0,0,N,0.0,0.0,0,52.0,-302.0,-501.0,-0.931358,103.293488,-0.018609,81.666667,88.250000,137.000000,85.250000,17.000000,37.025000,-0.081967,-0.327869,0.114754,-0.065574,0.000000,120.033333,0.060656,0.004471,1.726124,0.385000,5.210000,199.750000,53.040000,4.971600,1.366100,295.913000,-0.016216,0.000889,-0.009667,1.200000,0.000000,-0.004310,0.005394,0.396533,30.250000,-0.009689,4.00,4.000000,4.00,3.50,2.00,2.250000,3.50,2.000000,1.000000,4.00,8.00,14.250000,0.000000,0.000000,0.000000,0.016393,0.000000,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393,-0.341760,199.0,401,0
1,533,0,0,N,0.0,0.0,1,65.0,-44.0,-1023.0,-0.583333,73.500000,0.223395,130.471062,83.000000,141.333333,69.166667,17.333333,35.825000,0.021978,-0.043956,0.109890,0.000000,-0.003297,51.000000,-0.005495,0.002187,0.573115,-0.091506,-2.202054,149.000000,79.560000,4.539395,3.205620,182.941528,-0.016199,0.014305,-0.008014,0.681319,0.000000,0.021458,0.043421,-0.369212,28.666667,0.002114,1.00,1.333333,2.00,4.00,4.00,3.333333,4.00,3.333333,2.666667,3.00,3.00,21.333333,-0.010989,-0.010989,0.000000,0.000000,0.000000,-0.010989,0.010989,0.010989,0.010989,0.000000,-0.010989,0.010989,-0.965608,979.0,378,0
2,1064,0,0,N,0.0,0.0,1,66.0,-462.0,-550.0,0.076462,52.652720,-0.158016,82.666667,69.500000,122.000000,71.500000,20.500000,36.775000,0.028986,-0.144928,-0.086957,-0.057971,0.000000,80.700000,-0.020290,0.002810,2.034695,0.492000,4.466000,166.250000,70.720000,4.887540,1.919320,278.366400,-0.000993,-0.006957,0.018406,0.057971,0.000000,0.000374,0.001800,-0.258609,32.000000,-0.043752,0.75,2.750000,0.75,4.00,4.00,4.000000,4.00,4.000000,4.000000,3.75,1.50,24.000000,0.000000,-0.014493,-0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014493,-0.014493,0.000000,-0.569600,88.0,434,0
3,1736,1,0,N,0.0,1.0,2,41.0,-511.0,-570.0,0.000000,71.000000,0.045068,90.613842,81.571429,125.000000,82.000000,17.666667,36.900000,0.157895,-0.052632,0.355263,0.052632,0.001316,121.400000,-0.036770,0.004191,2.404000,0.464000,5.462000,182.405118,58.400000,4.549952,2.478876,317.882655,0.003896,0.001948,0.060390,0.968414,0.000000,-0.030289,-0.019621,0.641531,16.500000,-0.104167,2.50,3.000000,3.00,1.50,1.00,1.000000,0.50,0.500000,0.000000,3.50,5.50,4.500000,-0.020833,-0.041667,0.000000,-0.020833,0.000000,0.000000,-0.020833,-0.020833,0.000000,0.020833,-0.020833,-0.062500,-1.183528,59.0,542,0
4,2492,1,0,N,0.0,1.0,2,63.0,-209.0,-365.0,-0.235294,114.000000,-0.364164,114.042740,85.000000,140.000000,70.000000,16.000000,36.800000,-0.294118,-0.588235,-0.117647,0.000000,-0.017647,90.100000,0.000000,0.002976,2.297509,0.388903,2.152266,351.105529,66.300000,7.392401,0.899067,341.987231,0.014874,0.000550,0.003669,-0.207561,0.260000,-0.001873,-0.003815,-0.125311,30.000000,0.000000,4.00,4.000000,4.00,3.00,3.00,2.000000,3.00,3.000000,1.000000,3.00,8.00,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.00

##Select feature

In [ ]:
X_imp_surv = X_imp_surv[['Age', 'Gender', 'onset_site', 'onset_delta', 'diag_delta', 'diag_minus_onset',
        'mean_alsfrs_total', 'alsfrs_total_slope',
       'fvc_slope', 'fvc_mean', 'mean_Q1_Speech', 'mean_Q2_Salivation',
       'mean_Q3_Swallowing', 'mean_Q4_Handwriting', 'mean_Q5_Cutting',
       'mean_Q6_Dressing_and_Hygiene', 'mean_Q7_Turning_in_Bed',
       'mean_Q8_Walking', 'mean_Q9_Climbing_Stairs', 'mean_Q10_Respiratory',
       'mean_Bulbar', 'mean_motor', 'slope_Q1_Speech', 'slope_Q2_Salivation',
       'slope_Q3_Swallowing', 'slope_Q4_Handwriting', 'slope_Q5_Cutting',
       'slope_Q6_Dressing_and_Hygiene', 'slope_Q7_Turning_in_Bed',
       'slope_Q8_Walking', 'slope_Q9_Climbing_Stairs', 'slope_Q10_Respiratory',
       'slope_Bulbar', 'slope_motor','time_event','status']]

In [ ]:
X_imp_surv

,Age,Gender,onset_site,onset_delta,diag_delta,diag_minus_onset,mean_alsfrs_total,alsfrs_total_slope,fvc_slope,fvc_mean,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor,time_event,status
0,52.0,0,0,-501.0,-302.0,199.0,30.250000,-0.009689,-0.931358,103.293488,4.00,4.000000,4.00,3.50,2.00,2.250000,3.50,2.000000,1.000000,4.00,8.00,14.250000,0.000000,0.000000,0.000000,0.016393,0.000000,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393,401,0
1,65.0,0,1,-1023.0,-44.0,979.0,28.666667,0.002114,-0.583333,73.500000,1.00,1.333333,2.00,4.00,4.00,3.333333,4.00,3.333333,2.666667,3.00,3.00,21.333333,-0.010989,-0.010989,0.000000,0.000000,0.000000,-0.010989,0.010989,0.010989,0.010989,0.000000,-0.010989,0.010989,378,0
2,66.0,0,1,-550.0,-462.0,88.0,32.000000,-0.043752,0.076462,52.652720,0.75,2.750000,0.75,4.00,4.00,4.000000,4.00,4.000000,4.000000,3.75,1.50,24.000000,0.000000,-0.014493,-0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014493,-0.014493,0.000000,434,0
3,41.0,1,2,-570.0,-511.0,59.0,16.500000,-0.104167,0.000000,71.000000,2.50,3.000000,3.00,1.50,1.00,1.000000,0.50,0.500000,0.000000,3.50,5.50,4.500000,-0.020833,-0.041667,0.000000,-0.020833,0.000000,0.000000,-0.020833,-0.020833,0.000000,0.020833,-0.020833,-0.062500,542,0
4,63.0,1,2,-365.0,-209.0,156.0,30.000000,0.000000,-0.235294,114.000000,4.00,4.000000,4.00,3.00,3.00,2.000000,3.00,3.000000,1.000000,3.00,8.00,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,401,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,43.0,1,2,-673.0,-293.0,380.0,22.750000,-0.033444,0.678853,95.564652,3.00,3.750000,3.00,2.00,1.25,1.000000,2.00,2.000000,0.750000,4.00,6.00,9.000000,0.000000,-0.012821,0.000000,0.000000,-0.012821,0.000000,0.012821,0.000000,-0.012821,0.000000,0.000000,-0.025641,384,0
1646,70.0,1,1,-332.0,-181.0,151.0,29.500000,-0.090079,-0.270229,65.989389,1.25,1.500000,2.50,4.00,2.75,3.500000,4.00,3.750000,3.000000,3.25,3.75,21.000000,-0.014085,-0.028169,-0.014085,0.000000,-0.014085,-0.014085,0.014085,-0.014085,0.000000,0.014085,-0.028169,-0.042254,436,0
1647,46.0,0,1,-274.0,-212.0,62.0,20.500000,-0.055424,-0.608825,89.564790,1.50,3.250000,3.00,2.25,1.00,0.500000,2.25,2.000000,0.750000,4.00,4.50,8.750000,0.000000,0.011765,-0.011765,-0.035294,0.000000,0.000000,0.011765,0.000000,-0.011765,0.000000,-0.011765,-0.058824,410,0
1648,62.0,1,2,-288.0,-110.0,178.0,36.000000,0.000000,0.086856,138.802286,4.00,4.000000,4.00,4.00,4.00,3.000000,3.50,3.000000,2.500000,4.00,8.00,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.000000,0.000000,444,0


##Get dummy variable

In [ ]:
X_imp_surv_dummy = pd.get_dummies(X_imp_surv, columns = ['onset_site'],drop_first=True)
X_imp_surv_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1650 entries, 0 to 1649
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            1650 non-null   float64
 1   Gender                         1650 non-null   int64  
 2   onset_delta                    1650 non-null   float64
 3   diag_delta                     1650 non-null   float64
 4   diag_minus_onset               1650 non-null   float64
 5   mean_alsfrs_total              1650 non-null   float64
 6   alsfrs_total_slope             1650 non-null   float64
 7   fvc_slope                      1650 non-null   float64
 8   fvc_mean                       1650 non-null   float64
 9   mean_Q1_Speech                 1650 non-null   float64
 10  mean_Q2_Salivation             1650 non-null   float64
 11  mean_Q3_Swallowing             1650 non-null   float64
 12  mean_Q4_Handwriting            1650 non-null   f

In [ ]:
X_imp_surv_dummy.to_csv('_0415_features_without_nans_survival.csv')

In [ ]:
!ls

'_0411_all_features_with_nans(target included).csv'   sample_data
 _0415_features_without_nans_survival		      survival.csv
 _0415_features_without_nans_survival.csv


In [ ]:
files.download('_0415_features_without_nans_survival.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>